# Point & Block Kriging

<!-- SUMMARY: Implementations of Kriging for various targets: for Points, for regular blocks, for irregular blocks-->

<!-- CATEGORY: Methodology -->

This chapter describes the difference between Point Kriging, Regular Block Kriging and Irregular Block Kriging. It is delonstrated on a simulated 2-D data set.

## Import packages

In [ ]:
import numpy as np
import pandas as pd
import sys
import os
import matplotlib.pyplot as plt
import gstlearn as gl
import gstlearn.plot as gp
import gstlearn.document as gdoc

gdoc.setNoScroll()

# Creating the environmenet

We create a regular grid

In [ ]:
grid = gl.DbGrid.create(nx=[20,15],dx=[10.,10.])
ncell = grid.getSampleNumber()
grid

We create a Data Base of randomly located samples within a field covering the grid extension.

In [ ]:
coormin = grid.getCoorMinimum()
coormax = grid.getCoorMaximum()
nech = 100
data = gl.Db.createFromBox(nech, coormin, coormax)

We create a Model composed of a Nugget Effect and a Spherical components. This Model is used to simulate a variable at the sample locations.

In [ ]:
model = gl.Model()
model.addCovFromParam(gl.ECov.NUGGET, sill = 0.3)
model.addCovFromParam(gl.ECov.SPHERICAL, range=40., sill=0.7)
err = gl.simtub(None, dbout=data, model=model)
data

We define a Neighborhood. On purpose, this neighborhood is small (10 data maximum) to produce legible printouts.

In [ ]:
nmini = 1
nmaxi = 10
radius = 30.
nsect = 8
nsmax = 3
neigh = gl.NeighMoving.create(nmaxi=nmaxi, radius=radius, nmini=nmini, nsect=nsect, nsmax=nsmax)
neigh

# Point Kriging

In [ ]:
err = gl.kriging(data, grid, model, neigh, 
                 namconv=gl.NamingConvention("Point_Kriging"))

Two newly created variables have been added to the output Data Base (*grid*):
- one for the estimation (called *Point_Kriging.Simu.estim*)
- one for the standard deviation of the estimation error (called *Point_Kriging.simu.stdev*)

These two variables are stored as a consequance of the argument *flag_est* and *flag_std* which are not explicitely specified, but which are both defaulted to *True*.

In [ ]:
grid

Displaying the maps of the Point Estimation and the corresponding Standard Deviation of Estimation errors. Note that the estimation is performed at the nodes of the grid (center of each grid cell)... evan if the block *raster* is performed.

In [ ]:
ax = grid.plot("Point_Kriging*estim")
ax = data.plot(color="white")
ax.decoration(title="Point Kriging")

In [ ]:
ax = grid.plot("Point_Kriging*stdev")
ax = data.plot(color="white")
ax.decoration(title="Error for Point Kriging")

To better visualize the differences alongst the different kriging, we focus on a target grid cell (or node) and print out all the information used during the Kriging procedure. We focus on the **node** 

In [ ]:
node = 155
err = gl.krigtest(data, grid, model, neigh, node, calcul=gl.EKrigOpt.POINT)

## Standard Block Kriging

We perform an estimation of the average value of the variable over the cells (of the previous grid).

In [ ]:
ndiscs = [5,5]
err = gl.kriging(data, grid, model, neigh, calcul=gl.EKrigOpt.BLOCK, ndiscs=ndiscs,
                namconv=gl.NamingConvention("Block_Kriging"))

The output Db now contains the two additional variables *Block_Kriging.Simu.estim* and *Block_Kriging.Simu.stdev*.

In [ ]:
grid

Displaying the maps of the Point Estimation and the corresponding Standard Deviation of Estimation errors

In [ ]:
ax = grid.plot("Block_Kriging*estim")
ax = data.plot(color="white")
ax.decoration(title="Block Kriging")

In [ ]:
ax = grid.plot("Block_Kriging*stdev")
ax = data.plot(color="white")
ax.decoration(title="Error for Block Kriging")

We do not see too much difference between estimations, due to the small dimension of the grid cells, as it is demonstrated in the following scatter plot.

In [ ]:
ax = gp.correlation(grid, namex="Point_Kriging*estim", namey="Block_Kriging*estim", bins=100)
ax.decoration(title="Estimation", xlabel="Point", ylabel="Block")

In [ ]:
ax = gp.correlation(grid, namex="Point_Kriging*stdev", namey="Block_Kriging*stdev", bins=100)
ax.decoration(title="St. Dev.", xlabel="Point", ylabel="Block")

We check our reference target cell again. Remember that here the target is the average over the cell. In the final part of the printout, we clearly see the value of the cell extension i.e. 10 (important for future comparison).

In [ ]:
err = gl.krigtest(data, grid, model, neigh, node, calcul=gl.EKrigOpt.BLOCK, ndiscs=ndiscs)

## Irregular Block Kriging

Now we add two vectors in the Target Grid data base, which will contain the cell extension (variable per cell). Nevertheless, here, the cell extension is set to a constant value (for simplicity sake): this value (35) is different from the grid mesh (10). The variable block extension is assigned the locator *BLEX*.

In [ ]:
size = 35.
iuid = grid.addColumnsByConstant(1, size, "X-ext", gl.ELoc.BLEX, 0)
iuid = grid.addColumnsByConstant(1, size, "Y-ext", gl.ELoc.BLEX, 1)

In [ ]:
err = gl.krigcell(data, grid, model, neigh, ndiscs=ndiscs,
                  namconv=gl.NamingConvention("Irregular_Kriging"))

The two newly created results are added to the output data base *grid* with the radix *Irregular_Kriging".

In [ ]:
grid

Graphic display of the new resulting maps.

In [ ]:
ax = grid.plot("Irregular_Kriging*estim")
ax = data.plot(color="white")
ax.decoration(title="Block Kriging")

In [ ]:
ax = grid.plot("Irregular_Kriging*stdev")
ax = data.plot(color="white")
ax.decoration(title="Error for Block Kriging")

Comparison between standard Block Kriging and the Block Kriging with Irregular block size

In [ ]:
ax = gp.correlation(grid, namex="Block_Kriging*estim", namey="Irregular_Kriging*estim", bins=100)
ax.decoration(title="Estimation", xlabel="Regular Block", ylabel="Irregular Block")

In [ ]:
ax = gp.correlation(grid, namex="Block_Kriging*stdev", namey="Irregular_Kriging*stdev", bins=100)
ax.decoration(title="St. Dev.", xlabel="Regular Block", ylabel="Irregular Block")

We can now focus on our target block. Note that, unlike inn the case of regular block Kriging, the extension of the cell is now *35*.

In [ ]:
err = gl.krigtest(data, grid, model, neigh, node, calcul=gl.EKrigOpt.BLOCK, ndiscs=ndiscs,
                 flagPerCell = True)